In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib 
import time
from gtts import gTTS
import playsound
from keras.models import load_model

Using TensorFlow backend.
G:\Anaconda\envs\gputest\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
G:\Anaconda\envs\gputest\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
G:\Anaconda\envs\gputest\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
G:\Anaconda\envs\gputest\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
import warnings
warnings.filterwarnings("ignore")
import re
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow.keras.utils as ku 
import numpy as np

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [4]:
dialogue_corpus = open('dataset/dialogues_text.txt',encoding="utf8").read()

In [5]:
dialogue_corpus = re.sub(r'\[[0-9]*\]','',dialogue_corpus)
dialogue_corpus = re.sub(r'\d','',dialogue_corpus)
dialogue_corpus = re.sub(r'__eou__','',dialogue_corpus)
dialogue_corpus = re.sub(r'[?,!-+@#$%^&*\']','',dialogue_corpus)
dialogue_corpus = re.sub(r'\n','',dialogue_corpus)
dialogue_corpus = re.sub(r'\s+',' ',dialogue_corpus)

In [6]:
dialogue_corpus = dialogue_corpus.lower().split('.')

In [7]:
dialogue_corpus

['the kitchen stinks ',
 ' ill throw out the garbage ',
 ' so dick how about getting some coffee for tonight coffee i don ’ t honestly like that kind of stuff ',
 ' come on you can at least try a little besides your cigarette ',
 ' what ’ s wrong with that cigarette is the thing i go crazy for ',
 ' not for me dick ',
 ' are things still going badly with your houseguest getting worse ',
 ' now he ’ s eating me out of house and home ',
 ' i ’ ve tried talking to him but it all goes in one ear and out the other ',
 ' he makes himself at home which is fine ',
 ' but what really gets me is that yesterday he walked into the living room in the raw and i had company over that was the last straw ',
 ' leo i really think you ’ re beating around the bush with this guy ',
 ' i know he used to be your best friend in college but i really think it ’ s time to lay down the law ',
 ' you ’ re right ',
 ' everything is probably going to come to a head tonight ',
 ' i ’ ll keep you informed ',
 ' would 

In [8]:
tokenizer = Tokenizer()

In [9]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [10]:
data = []
common_words = get_top_n_words(dialogue_corpus, 3000)
for word, freq in common_words:
    data.append(word)

In [11]:
tokenizer.fit_on_texts(data)

In [12]:
word_index = tokenizer.word_index

In [13]:
total_num_words = len(word_index) + 1

In [14]:
input_sequences = []
for line in dialogue_corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [15]:
max_sequence_len = max([len(x) for x in input_sequences])

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_num_words)

In [16]:
from tensorflow.keras.layers import Embedding, CuDNNLSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [17]:
model = Sequential()
model.add(Embedding(total_num_words, 100, input_length=max_sequence_len-1))
model.add(CuDNNLSTM(150, return_sequences = True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(100))
model.add(Dense(total_num_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_num_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 162, 100)          300100    
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (None, 162, 150)          151200    
_________________________________________________________________
dropout (Dropout)            (None, 162, 150)          0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100)               100800    
_________________________________________________________________
dense (Dense)                (None, 1500)              151500    
_________________________________________________________________
dense_1 (Dense)      

In [18]:
history = model.fit(predictors, label, epochs=10, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
894640/894640 [==============================] - 820s 916us/sample - loss: 5.5781
Epoch 2/10
894640/894640 [==============================] - 820s 916us/sample - loss: 5.1144
Epoch 3/10
894640/894640 [==============================] - 835s 933us/sample - loss: 4.9762
Epoch 4/10
894640/894640 [==============================] - 850s 950us/sample - loss: 4.8963
Epoch 5/10
894640/894640 [==============================] - 842s 942us/sample - loss: 4.8414
Epoch 6/10
894640/894640 [==============================] - 829s 926us/sample - loss: 4.8030
Epoch 7/10
894640/894640 [==============================] - 824s 921us/sample - loss: 4.7749
Epoch 8/10
894640/894640 [==============================] - 823s 920us/sample - loss: 4.7540 ETA: 0s 
Epoch 9/10
894640/894640 [==============================] - 828s 926us/sample - loss: 4.7381
Epoch 10/10
894640/894640 [==============================] - 1767s 2ms/sample - loss: 4.7253


In [30]:
capture = cv2.VideoCapture(0)
words_list = ['HELLO','I','HOW','YOU','*','+']
words_row = [10,150,280,430,580,615]
list_1 = ['A','B','C','D','E','F','2','#','G','H','I','J','K','L','M','=']
list_2 = ['N','O','P','Q','R','S','1','*','T','U','V','W','X','Y','Z','+']
alphabets = ['A','B','C','D','E','F','2','#','G','H','I','J','K','L','M','=','N','O','P','Q','R','S','1','*','T','U','V','W','X','Y','Z','+']
row_1 = [10,50,90,130,170,210,250,290,330,370,410,450,490,530,570,610]
up_row = True
word_counter = 0
set_time = 0
curr_alpha = 'A'
alpha_counter = 0
sentence = ''
words = ''
last_blink = 0
change = False
is_first_letter = True
new_word = ''

while True:      
    ret,frame = capture.read()
    
    for i in range(len(words_list)):
        cv2.putText(frame,text=words_list[i],org=(words_row[i],330),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,255,0),thickness=1,lineType=cv2.LINE_AA)
        if i < 5:
            cv2.rectangle(frame,(words_row[i]-5,300),(words_row[i+1]-10,340),(0,255,0),2)
        else:
            cv2.rectangle(frame,(words_row[i]-5,300),(640,340),(0,255,0),2)
            
    for i,j in zip(row_1,range(16)):
        cv2.rectangle(frame,(i,350),(i+30,380),(0,255,0),2)
        cv2.putText(frame,text=list_1[j],org=(i+7,370),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,255,0),thickness=1,lineType=cv2.LINE_AA)
        cv2.rectangle(frame,(i,390),(i+30,420),(0,255,0),2)
        cv2.putText(frame,text=list_2[j],org=(i+7,410),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,255,0),thickness=1,lineType=cv2.LINE_AA)
    
    if up_row:
        if time.time() - set_time > 3:
            if word_counter < len(words_list):
                curr_word = words_list[word_counter]
                word_counter += 1
                set_time = time.time()
            else:
                word_counter = 0
                curr_word = words_list[0]
                set_time = time.time()
        if word_counter == 0:
            cv2.putText(frame,text=curr_word,org=(words_row[0],330),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,0,0),thickness=1,lineType=cv2.LINE_AA)
        else:
            cv2.putText(frame,text=curr_word,org=(words_row[word_counter-1],330),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,0,0),thickness=1,lineType=cv2.LINE_AA)

    else:
        if time.time() - set_time > 1:
            if alpha_counter < len(alphabets):
                curr_alpha = alphabets[alpha_counter]
                alpha_counter += 1
                set_time = time.time()
            else:
                alpha_counter = 0
                curr_alpha = 'A'
                set_time = time.time()
        
        if curr_alpha in list_1:
            if alpha_counter == 0:
                cv2.putText(frame,text=curr_alpha,org=(row_1[0]+7,370),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,0,255),thickness=1,lineType=cv2.LINE_AA)
            else:
                cv2.putText(frame,text=curr_alpha,org=(row_1[alpha_counter-1]+7,370),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,0,255),thickness=1,lineType=cv2.LINE_AA)
        if curr_alpha in list_2 and (alpha_counter-17) >= 0:
            cv2.putText(frame,text=curr_alpha,org=(row_1[alpha_counter-17]+7,410),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,0,255),thickness=1,lineType=cv2.LINE_AA)   
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:
        
        landmarks = predictor(gray,face)
        
        le_y1_top = landmarks.part(37).y
        le_y1_bottom = landmarks.part(41).y
        le_y2_top = landmarks.part(38).y
        le_y2_bottom = landmarks.part(40).y
        
        re_y1_top = landmarks.part(43).y
        re_y1_bottom = landmarks.part(47).y
        re_y2_top = landmarks.part(44).y
        re_y2_bottom = landmarks.part(46).y
        
        if (time.time() - last_blink) > 2:
            if abs(re_y2_top-re_y2_bottom)  < 6:
                if up_row:
                    if curr_word == '*':
                        alpha_counter = 0
                        up_row = False
                        words += ' '
                    elif curr_word == '+':
                        tts = gTTS(text=words,lang='en-in')
                        words = ''
                        filename =  str(time.time()) + '.mp3'
                        tts.save(filename)
                        playsound.playsound(filename)
                    else:   
                        sentence += ' ' + curr_word
                        words += ' ' + curr_word
                        token_list = tokenizer.texts_to_sequences([sentence])[0]
                        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
                        array = model.predict(token_list)
                        array = np.argpartition(array[0], len(array[0]) - 4)[-4:]
                        for i,j in zip(array,range(len(array))):
                            words_list[j] = data[i-1].upper()
                        word_counter = 0
                else:
                    if curr_alpha == '*' or curr_alpha == '#':
                        word_counter = 0
                        up_row = True
                    elif curr_alpha == '=':
                        words += ' '
                        sentence += ' ' + new_word
                        new_word = ''
                        token_list = tokenizer.texts_to_sequences([sentence])[0]
                        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
                        array = model.predict(token_list)
                        array = np.argpartition(array[0], len(array[0]) - 4)[-4:]
                        for i,j in zip(array,range(len(array))):
                            words_list[j] = data[i-1].upper()
                    elif curr_alpha == '1':
                        alpha_counter = 0
                    elif curr_alpha == '2':
                        alpha_counter = 16
                    elif curr_alpha == '+':
                        tts = gTTS(text=words,lang='en-in')
                        words = ''
                        filename =  str(time.time()) + '.mp3'
                        tts.save(filename)
                        playsound.playsound(filename)
                        
                    else:
                        new_word += curr_alpha.lower()
                        words += curr_alpha
                last_blink = time.time()
    
    
    if len(sentence) != 0:
        cv2.putText(frame,text=sentence[-1],org=(70,100),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,0,255),thickness=1,lineType=cv2.LINE_AA)
    
    if up_row:
        cv2.putText(frame,text=curr_word,org=(140,100),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,255,0),thickness=1,lineType=cv2.LINE_AA)
    else:
        cv2.putText(frame,text=curr_alpha,org=(210,100),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,255,0),thickness=1,lineType=cv2.LINE_AA)
        
    cv2.putText(frame,text=words,org=(10,280),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,0),thickness=1,lineType=cv2.LINE_AA)
    
    
    
    
    cv2.imshow('frame',frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
capture.release() 
cv2.destroyAllWindows()